In [5]:
import pandas as pd

path = r"C:\Users\joacosta\Videos\alarmsClean.csv"

df = pd.read_csv(path)

df.head(10)

,timeStamp,Part name,Alarm text
0,20/09/2025 10:26:08,CA341,Disabled
1,20/09/2025 10:26:08,CA340,Disabled
2,20/09/2025 10:31:23,CA444,Disabled
3,20/09/2025 10:42:48,CA521,Disabled
4,20/09/2025 10:47:59,CA007,Disabled
5,20/09/2025 10:47:59,CA007,Item on bellows is active
6,20/09/2025 10:48:41,CA183,Disabled
7,20/09/2025 10:48:41,CA183,Item on bellows is active
8,20/09/2025 10:51:05,CA183,Disabled
9,20/09/2025 10:51:05,CA183,Item on bellows is active


In [6]:
# Group by Part name and count alarms
carrier_alarm_counts = df.groupby('Part name').size().reset_index(name='total_alarms')

print("Total alarms per carrier:")
print(carrier_alarm_counts)

# Group by Part name and Alarm text to see breakdown
detailed_breakdown = df.groupby(['Part name', 'Alarm text']).size().reset_index(name='count')
print("\nDetailed breakdown by carrier and alarm type:")
print(detailed_breakdown)

# Filter for specific alarm types and count
target_alarms = ['Item on bellows is active', 'Disabled']
filtered_alarms = df[df['Alarm text'].isin(target_alarms)]

# Group by Part name for the filtered alarms
target_alarm_counts = filtered_alarms.groupby('Part name').size().reset_index(name='target_alarm_count')

print(f"\nCarriers with 'Item on bellows is active' or 'Disabled' alarms:")
print(target_alarm_counts)

# Total sum of target alarms
total_target_alarms = target_alarm_counts['target_alarm_count'].sum()
print(f"\nTotal sum of 'Item on bellows is active' and 'Disabled' alarms: {total_target_alarms}")

# Show which carriers have which specific alarm types
print(f"\nBreakdown by specific alarm type:")
for alarm_type in target_alarms:
    carriers_with_alarm = df[df['Alarm text'] == alarm_type]['Part name'].value_counts()
    if not carriers_with_alarm.empty:
        print(f"\n'{alarm_type}' alarms:")
        print(carriers_with_alarm)

Total alarms per carrier:
   Part name  total_alarms
0      CA007            22
1      CA025             2
2      CA037            16
3      CA061             2
4      CA071             2
5      CA079            20
6      CA103             6
7      CA117             2
8      CA169             8
9      CA183            28
10     CA213             2
11     CA223             2
12     CA299             4
13     CA340             1
14     CA341             1
15     CA343            12
16     CA375             2
17     CA407             2
18     CA444             1
19     CA493             2
20     CA495             2
21     CA513             5
22     CA521             1
23     CA591             2

Detailed breakdown by carrier and alarm type:
   Part name                 Alarm text  count
0      CA007                   Disabled     11
1      CA007  Item on bellows is active     11
2      CA025                   Disabled      1
3      CA025  Item on bellows is active      1
4      CA037     

In [7]:
# Create a comprehensive summary
summary = df.groupby('Part name').agg({
    'Alarm text': ['count', lambda x: (x == 'Disabled').sum(), lambda x: (x == 'Item on bellows is active').sum()]
}).round(2)

# Flatten column names
summary.columns = ['total_alarms', 'disabled_count', 'bellows_active_count']
summary['target_alarms_total'] = summary['disabled_count'] + summary['bellows_active_count']
summary = summary.reset_index()

print("Comprehensive summary:")
print(summary)

# Extract carrier numbers for further analysis if needed
summary['carrier_number'] = summary['Part name'].str.extract(r'CA(\d+)')
summary['carrier_formatted'] = summary['carrier_number'].apply(lambda x: f"{int(x):04d}" if pd.notna(x) else None)

print(f"\nCarriers with target alarms (formatted): {summary[summary['target_alarms_total'] > 0]['carrier_formatted'].tolist()}")

Comprehensive summary:
   Part name  total_alarms  disabled_count  bellows_active_count  \
0      CA007            22              11                    11   
1      CA025             2               1                     1   
2      CA037            16               8                     8   
3      CA061             2               1                     1   
4      CA071             2               1                     1   
5      CA079            20              10                    10   
6      CA103             6               3                     3   
7      CA117             2               1                     1   
8      CA169             8               4                     4   
9      CA183            28              14                    14   
10     CA213             2               1                     1   
11     CA223             2               1                     1   
12     CA299             4               2                     2   
13     CA340             

In [8]:
# Count alarms by alarm text (overall frequency)
alarm_text_counts = df['Alarm text'].value_counts()

print("Alarm frequency by text:")
print(alarm_text_counts)

# Show as a percentage of total alarms
alarm_text_percentage = df['Alarm text'].value_counts(normalize=True) * 100

print(f"\nAlarm frequency by text (percentage):")
print(alarm_text_percentage.round(2))

# Create a summary dataframe for better visualization
alarm_summary = pd.DataFrame({
    'alarm_text': alarm_text_counts.index,
    'count': alarm_text_counts.values,
    'percentage': alarm_text_percentage.round(2).values
})

print(f"\nSummary table:")
print(alarm_summary)

# Total number of alarms
total_alarms = len(df)
print(f"\nTotal alarms in dataset: {total_alarms}")

Alarm frequency by text:
Alarm text
Disabled                     75
Item on bellows is active    71
Item on activated carrier     1
Name: count, dtype: int64

Alarm frequency by text (percentage):
Alarm text
Disabled                     51.02
Item on bellows is active    48.30
Item on activated carrier     0.68
Name: proportion, dtype: float64

Summary table:
                  alarm_text  count  percentage
0                   Disabled     75       51.02
1  Item on bellows is active     71       48.30
2  Item on activated carrier      1        0.68

Total alarms in dataset: 147
